In [1]:
import substra
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from pathlib import Path


In [2]:
client = substra.Client.from_config_file("node_A")
cp_key = "be1cf90d-929f-4a74-bbd4-d212272f073c"
test_data_path = Path('/') / 'home' / 'user' / 'data' / 'test'


In [3]:
traintuples = client.list_traintuple(filters=[f'traintuple:compute_plan_key:{cp_key}'])


In [4]:
rank_max = 0
tt_key = ""

for tt in traintuples:
    if tt.rank > rank_max:
        rank_max = tt.rank
        tt_key = tt.key

In [6]:

# get the last traintuple of the compute plan
traintuple = client.get_traintuple(tt_key)

# Download the associated model
model_path = Path(".")
model_filename = f'model_{traintuple.train.models[0].key}'
client.download_model_from_traintuple(traintuple.key, folder=model_path)

# SPECIFIC TO YOUR ALGO
# -----------------------
#
# Here we are loading the model that your algo produced
# so if you change the algo, for example if you use PyTorch,
# then you might need to change this part as well.

# Load the model and create predictions: this code depends on the algo code
model = keras.models.load_model(str(model_path / model_filename))

image_size = (180, 180)
batch_size = 32
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=test_data_path,
    labels="inferred",
    label_mode="binary",
    shuffle=False,
    seed=0,
    image_size=image_size,
    batch_size=batch_size,
)

predictions = np.array([])
labels = np.array([])
for x, y in test_ds:
    predictions = np.concatenate([predictions, model.predict(x).ravel()])
    labels = np.concatenate([labels, y.numpy().ravel()])
file_paths = test_ds.file_paths

df_submission = pd.DataFrame(data={
    'file_paths': file_paths,
    'predictions': predictions
})
df_submission["file_paths"] = df_submission["file_paths"].apply(
    lambda x: x.replace("/home/user/data/test", "/data/challenges_data/test"))



2021-11-12 17:26:18.855307: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 17:26:18.868754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 17:26:18.869796: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-12 17:26:18.871428: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 6000 files belonging to 2 classes.


2021-11-12 17:26:23.702354: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-12 17:26:26.430373: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8204


NameError: name '__file__' is not defined

In [7]:
submission_filepath = Path('.') / 'df_submission.csv'
df_submission.to_csv(submission_filepath, index=False)

print(df_submission.head())

print(f"Created the file at {submission_filepath}")

# Delete the downloaded model
(model_path / model_filename).unlink()

                                          file_paths  predictions
0  /data/challenges_data/test/target_0/normal_0_1...     0.026310
1  /data/challenges_data/test/target_0/normal_0_1...     0.010726
2  /data/challenges_data/test/target_0/normal_0_1...     0.129540
3  /data/challenges_data/test/target_0/normal_0_1...     0.043004
4  /data/challenges_data/test/target_0/normal_0_1...     0.271591
Created the file at df_submission.csv
